<a href="https://colab.research.google.com/github/AlanIslasMorris/Fusi-n-Agro/blob/main/Suministro_y_venta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [56]:
import pandas as pd

In [57]:
#Zona
df3 = pd.read_excel('Zona.xlsx').groupby(['Empresa','RM/ REIQ_','País_3'])['Rtnm'].count().reset_index().rename(columns = {'RM/ REIQ_':'RM'})

#Presentación a volumen
df2 = pd.read_excel('Presentacion_a_volumen.xlsx').groupby('Presentación')['Vol aux'].mean().reset_index()

#Forecast JUNIO
df = pd.read_excel('Forecast.xlsx').groupby(['RM','Producto','Item a producto.Item','Presentación','Mes'])['Volumen'].sum().reset_index()
df = df.rename(columns={'Item a producto.Item':'ITEMNMBR'})
#Forecast por empresa
df = df.merge(df3, on='RM', how ='left')
df = df.groupby(['Empresa','Producto','ITEMNMBR','Presentación','Mes'])['Volumen'].sum().reset_index()
df = df.merge(df2,on='Presentación', how = 'left')
# Litros a Unidades
df['Unidades']= df['Volumen']/df['Vol aux']
#MX, junio
df = df[(df['Empresa'] == 'México')&(df['Mes'] == 7)]
df = df.rename(columns={'Unidades':'Unidades julio'})

# Pedidos
df1 = pd.read_excel('Pedidos.xlsx',skiprows=2).groupby(['EMPRESA', 'ITEMNMBR'])['Sum of PEDIDOS'].sum().reset_index()
df1 = df1[df1['EMPRESA'] == 'MX']
#Selecciona valores superiores a 0
df1=df1[df1['Sum of PEDIDOS']>0].sort_values(by='Sum of PEDIDOS', ascending = False)

# Inventario
df7 = pd.read_excel('Pedidos.xlsx',skiprows=2).groupby(['EMPRESA', 'ITEMNMBR'])['Sum of INVENTARIO'].sum().reset_index()
df7 = df7[df7['EMPRESA'] == 'MX']
#Selecciona valores superiores a 0
df7=df7[df7['Sum of INVENTARIO']>0].sort_values(by='Sum of INVENTARIO', ascending = False)



#Pendiente
df4 = pd.read_excel('Pendiente.xlsx',skiprows=2)
df4 = df4.rename(columns={"Sum of PENDIENTE_R":"Sum of PENDIENTE"})



In [58]:
df4 = df4.groupby(["EMPRESA","ITEMNMBR"])["Sum of PENDIENTE"].sum().reset_index()
df4 = df4[df4['EMPRESA'] == 'MX']



#Volumen de ventas
df5 = pd.read_excel('Volumen.xlsx',skiprows=2).groupby(["Empresa","ID_PRODUCTO","'Base'[Mes]"])["Ver selección"].sum().reset_index()
df5 = df5.rename(columns={"'Base'[Mes]":"Mes",
                          "ID_PRODUCTO":"ITEMNMBR",
                          "Ver selección":"Volumen ventas junio"})
df5 = df5[(df5['Empresa'] == 'MX')&(df5['Mes'] == 6)]


#Forecast Mayo
df6 = pd.read_excel('Forecast.xlsx').groupby(['RM','Producto','Item a producto.Item','Presentación','Mes'])['Volumen'].sum().reset_index()
df6 = df6.rename(columns={'Item a producto.Item':'ITEMNMBR'})

#Forecast por empresa
df6 = df6.merge(df3, on='RM', how ='left')
df6 = df6.groupby(['Empresa','Producto','ITEMNMBR','Presentación','Mes'])['Volumen'].sum().reset_index()
df6 = df6.merge(df2,on='Presentación', how = 'left')
# Litros a Unidades
df6['Unidades']= df6['Volumen']/df6['Vol aux']
#MX, julio
df6 = df6[(df6['Empresa'] == 'México')&(df6['Mes'] == 8)]
df6 = df6.rename(columns={'Unidades':'Unidades agosto'})



In [59]:
merged_df = df.merge(df1[['ITEMNMBR', 'Sum of PEDIDOS']], on='ITEMNMBR', how='left')

merged_df = merged_df.merge(df4[['ITEMNMBR', 'Sum of PENDIENTE']], on='ITEMNMBR', how='left')

merged_df = merged_df.merge(df5[['ITEMNMBR', 'Volumen ventas junio']], on='ITEMNMBR', how='left')

merged_df = merged_df.merge(df6[['ITEMNMBR', 'Unidades agosto']], on='ITEMNMBR', how='left')

merged_df = merged_df.merge(df7[['ITEMNMBR', 'Sum of INVENTARIO']], on='ITEMNMBR', how='left')


In [60]:
from pandas.core.reshape.merge import merge_ordered
# Cálculo de cantidad disponible
merged_df['Disponible']=merged_df['Sum of INVENTARIO']\
                        +merged_df['Sum of PENDIENTE']\
                        - merged_df['Sum of PEDIDOS']

# Compra sugerida
#++++++++++++++++++++Actualizar el mes++++++++++++++++++++++++++++++++++++++++++
merged_df['Compra_sugerida'] = merged_df['Unidades agosto']\
                              -merged_df['Disponible']
merged_df = merged_df.sort_values(by='Unidades agosto', ascending=False)


In [61]:
merged_df.to_csv('Tabla de compras_.csv', index=False)